In [89]:
import threading
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import webbrowser
import numpy as np

### Introduction
**Model of electric engine.**

We can regulate engine to operate in range 1000 - 5000 RPMs.

Differential equation describing the system.

$$J\frac{d\omega}{dt} = M_e - M_0 - M_{LOAD}$$
where:
- $M_e$: electromagnetic moment
- $M_0$: braking moment
- $M_{LOAD}$: load moment
- $I$: moment of inertia

Transformation to differential form using the Eulerian method.
$$I\frac{d\omega(t+\Delta t) - \omega (t)}{\Delta t} = M_e(t)-M_0-M_{LOAD}(t)$$
$$\omega (t+\Delta t) = \omega(t) + \frac{\Delta t}{I}(M_e(t)- M_0-M_{LOAD}(t))$$

What about $M_e(t)$?
$$U_{PID}(n) = K_p \cdot e(n) + K_i \sum^t_{k=0}e(k)\Delta t + K_d \cdot \frac{e(n) - e(n-1)}{\Delta t}$$
$$M_e(n) = U_{PID} * const$$
$$e(n) = \omega_{ref} - \omega(n)$$

### Defined Parameters

In [90]:
# Parameters of simulation
referencedRevolutionsPerMinute = 3000

# Simulation time parameters
timeOfSimulation = 3600
timeOfSample = 0.1

# Parameters of crankshaft
brakingMoment = 1
loadMoment = 3
momentOfInertia = 1.2
constantOfElectromagneticMoment = 0.4

# Parameters of PI regulator
Kp = 0.007
Ti = 50
Td = 2

# Constraints
Umax = 24
Umin = 0


# Lists of measured values
timeOfSimulationList = [0.0]
loadMomentList = [0.0]
electromagneticMomentList = [0.0]
adjustmentErrors = [referencedRevolutionsPerMinute]
voltagesList = [0.0]
revolutionsList = [0]
previousrevolutionsList = [0]
previoustimeOfSimulationList = [0]

### Calculations

In [91]:
def calculateNumberOfIterations(timeOfSimulation: int, timeOfSample: float) -> int:
    """ Calculates number of iterations for simulation of process

        @Parameters:
        - timeOfSimulation (int): total time of simulation in seconds
        - timeOfSample (float): time at which we repeat the measurement in seconds

        @Return:
        - int: number of iterations
    """
    return int(timeOfSimulation / timeOfSample) + 1

In [92]:
def calculateAdjustmentError(referencedRevolutionsPerMinute: float, currentRevolutionsPerMinute: float) -> float:
    """ Calculates adjustment error which is difference between referenced value and current one

        @Parameters:
        - referencedRevolutionsPerMinute (float): set value to be obtained by regulator
        - currentRevolutionsPerMinute (float): current value

        @Return:
        - float: error
    """
    return referencedRevolutionsPerMinute - currentRevolutionsPerMinute

In [93]:
def calculateVoltageOfRegulator(errorList: list[float], iteration: int) -> float:
    """ Calculates current voltage of regulator using PID control.

    @Parameters:
    - errorList (list[float]): list of errors at the moment and before
    - iteration (int): information about current simulation iteration

    @Return:
    - float: current voltage of regulator
    """

    proportional = Kp * errorList[iteration]
    
    integral = Kp * sum(errorList) * timeOfSample / Ti

    # Deriative part can be done from second iteration
    if iteration > 0:  
        derivative = (errorList[iteration] - errorList[iteration - 1]) / timeOfSample
    else:
        derivative = 0.0 

    derivative = Kp * (Td/timeOfSample) * derivative

    voltage = proportional + integral + derivative

    return voltage


In [94]:
def calculateElectromagneticMoment(constant : float, currentVoltage : float) -> float:
    """ Calculates current electromagnetic moment based on voltage of regulator

        @Parameters:
        - constant (float): used to scale moment 
        - currentVoltageOfRegulator (float): voltage of regulator at the moment

        @Return
        - float: current electromagnetic moment
    """
    return constant * currentVoltage

In [95]:
def calculateNormalizedVoltage(voltgeOfRegulator : float) -> float:
    """ Calculates normalized voltage based on predefined constraints <Umin;Umax> [V]

    @Parameters:
    - voltageOfRegulator (float): current voltage of regulator

    @Return:
    - float: normalized voltage used to create electromagnetic moment
    """
    return max(Umin, min(Umax, voltgeOfRegulator))

In [96]:
def convertToAngularVelocity(valueToBeConverted : float) -> float:
    return valueToBeConverted * (2 * np.pi / 60)

In [97]:
def convertToRevolutionsPerMinute(valueToBeConverted : float) -> float:
    return valueToBeConverted * (60 / (2 * np.pi))

In [98]:
def calculateRevolutions(latestRevolution : float, latestElectromagneticMoment : float) -> float:
    """ Calculates the updated revolutions per minute (RPM) based on the system's moments.

        @Parameters:
        - latestRevolution (float): value of revolution in previous iteration
        - latestElectromagneticMoment (float): value of electromagnetic moment in previous iteration

        @Return:
        - float: updated revolutions
    """
    omega = convertToAngularVelocity(latestRevolution)
    acceleration = (
        latestElectromagneticMoment - loadMoment - brakingMoment) / momentOfInertia
    newOmega = omega + timeOfSample * acceleration

    return convertToRevolutionsPerMinute(newOmega)

### Visualizations

In [99]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Load Moment"),
        dcc.Slider(
            id='slider-loadMoment', min=1, max=5, step=0.1, value=loadMoment,
            marks={i: str(i) for i in range(1, 11)}
        ),
        html.Label("Referenced RPMs"),
        dcc.Slider(
            id='slider-referencedRevolutionsPerMinute', min=1000, max=5000,step=1000,value=referencedRevolutionsPerMinute,
            marks={i: str(i*1000) for i in range(1, 6)}
        ),
        html.Label("Proportional gain (Kp)"),
        dcc.Slider(
            id='slider-Kp', min=0.001, max=0.02, step=0.001, value=Kp,
            marks={round(i, 3): str(round(i, 3))
                   for i in [0.001, 0.005, 0.01, 0.015, 0.02]}
        ),
        html.Label("Integral time (Ti)"),
        dcc.Slider(
            id='slider-Ti',
            min=10, max=50, step=5, value=Ti,
            marks={round(i, 9): str(round(i, 9))
                   for i in [10, 15, 20, 25, 30, 35,40,45,50]}
        ),

        html.Label("Derivative time (Td)"),
        dcc.Slider(
            id='slider-Td',
            min=0, max=30, step=5, value=Td,
            marks={round(i, 7): str(round(i, 7))
                   for i in [0, 5, 10,15,20,25,30]}
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    html.Div([
        dcc.Graph(id='moments-graph'),
        dcc.Graph(id='revolutions-graph')
    ]),
])


@app.callback(
    [
        Output('moments-graph', 'figure'),
        Output('revolutions-graph', 'figure')
    ],
    [
        Input('slider-loadMoment', 'value'),
        Input('slider-referencedRevolutionsPerMinute', 'value'),
        Input('slider-Kp', 'value'),
        Input('slider-Ti', 'value'),
        Input('slider-Td', 'value'),
    ]
)

def updateGraphs(newLoadMoment, newReferencedRPM, newKp, newTi, newTd):
    # Update global parameters
    global loadMoment, referencedRevolutionsPerMinute, Kp, Ti, Td
    loadMoment = newLoadMoment
    referencedRevolutionsPerMinute = newReferencedRPM
    Kp = newKp
    Ti = newTi
    Td = newTd

    # Reinitialize global lists
    global timeOfSimulationList, loadMomentList, electromagneticMomentList
    global adjustmentErrors, voltagesList, revolutionsList, brakingMomentList,previousrevolutionsList,previoustimeOfSimulationList
    timeOfSimulationList = [0.0]
    loadMomentList = [0.0]
    electromagneticMomentList = [0.0]
    adjustmentErrors = [referencedRevolutionsPerMinute]
    voltagesList = [0.0]
    revolutionsList = [0]
    brakingMomentList = [brakingMoment]


    def dynamicLoadMoment(time, totalSimulationTime):
        oneThirdTime = totalSimulationTime / 3
        if time < oneThirdTime:
            return 1.0 
        elif time < 2 * oneThirdTime:
            return 5.0 
        else:
            return 2.0


    # Simulation
    for i in range(int(timeOfSimulation / timeOfSample)):
        timeOfSimulationList.append(timeOfSimulationList[i] + timeOfSample)

        # Dynamiczne obciążenie
        loadMoment = dynamicLoadMoment(timeOfSimulationList[-1], timeOfSimulation)
        loadMomentList.append(loadMoment)

        voltage = calculateNormalizedVoltage(
            calculateVoltageOfRegulator(adjustmentErrors, i)
        )
        voltagesList.append(voltage)

        electromagneticMoment = calculateElectromagneticMoment(
            constantOfElectromagneticMoment, voltagesList[i]
        )
        electromagneticMomentList.append(electromagneticMoment)

        revolutions = calculateRevolutions(
            revolutionsList[i], electromagneticMomentList[i])
        revolutionsList.append(revolutions)

        adjustmentError = calculateAdjustmentError(
            referencedRevolutionsPerMinute, revolutionsList[i]
        )
        adjustmentErrors.append(adjustmentError)

        brakingMomentList.append(brakingMoment)



# Revolutions graph
    revolutionsFigure = go.Figure()

    revolutionsFigure.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=revolutionsList,
        mode='lines',
        name='Revolutions',
        hovertemplate='Time: %{x}<br>Revolutions: %{y:.0f}<extra></extra>'
    ))

    revolutionsFigure.add_trace(go.Scatter(
        x=previoustimeOfSimulationList,
        y=previousrevolutionsList,
        mode='lines',
        name='Previous Revolutions',
        line=dict(color='gray', dash="dash"),
        hovertemplate='Time: %{x}<br>Previous revolutions: %{y:.0f}<extra></extra>'
    ))

    revolutionsFigure.add_hline(y=referencedRevolutionsPerMinute, line_dash="dot",
                                annotation_text="Target RPM")
    revolutionsFigure.update_layout(
        title="Revolutions Over Time",
        xaxis_title="Time (s)",
        yaxis_title="Revolutions per Minute (RPM)"
    )
    previousrevolutionsList = revolutionsList
    previoustimeOfSimulationList = timeOfSimulationList

    # Moments graph (Load, Electromagnetic, and Braking Moment)
    momentsFigure = go.Figure()

    momentsFigure.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=loadMomentList,
        mode='lines',
        name='Load Moment',
        line=dict(color='blue'),
        hovertemplate='Time: %{x}<br>Load Moment: %{y:.1f}<extra></extra>'
    ))

    momentsFigure.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=electromagneticMomentList,
        mode='lines',
        name='Electromagnetic Moment',
        line=dict(color='red'),
        hovertemplate='Time: %{x}<br>Load Moment: %{y:.1f}<extra></extra>'
    ))

    momentsFigure.add_trace(go.Scatter(
        x=timeOfSimulationList,
        y=brakingMomentList,
        mode='lines',
        name='Braking Moment',
        line=dict(color='green'),
        hovertemplate='Time: %{x}<br>Load Moment: %{y:.1f}<extra></extra>'
    ))
    timeInMinutes = [round(t / 60, 2) for t in timeOfSimulationList]
    momentsFigure.update_layout(
    title=f"Load, Electromagnetic, and Braking Moment ({brakingMoment}) Over Time",
    xaxis_title="Time (s)",  
)

    
    return revolutionsFigure, momentsFigure


def openBrowser():
    """Open the web browser to the Dash app"""
    webbrowser.open("http://127.0.0.1:8050")

if __name__ == '__main__':
    threading.Thread(target=lambda: app.run_server(
        debug=True, use_reloader=False, host='127.0.0.1', port=8050)).start()

    openBrowser()

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[99], line 104, in updateGraphs(
    newLoadMoment=3,
    newReferencedRPM=3000,
    newKp=0.007,
    newTi=50,
    newTd=2
)
     98 voltage = calculateNormalizedVoltage(
     99     calculateVoltageOfRegulator(adjustmentErrors, i)
    100 )
    101 voltagesList.append(voltage)
    103 electromagneticMoment = calculateElectromagneticMoment(
--> 104     constantOfElectromagneticMoment, voltagesList[i]
        electromagneticMoment = 2.0004506413363665
        voltagesList = [0.0, 21.042, 21.084, 23.36533233099479, 23.418517740395927, 14.04771924972819, 14.035066154011972, 13.019325754547829, 12.996322440061933, 17.193804670166134, 17.220968556413336, 17.697369594992974, 17.731389014586846, 15.87521021115213, 15.877351401545727, 15.678179228482566, 15.676014738922737, 16.520273959922854, 16.536349660810988, 16.64253361275269, 16.6